In [54]:
import re
import nltk
import random
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 

import numpy as np
import pandas as pd
import heapq
import numpy as np
from collections import Counter

# Preparing DataFrame

In [55]:
podcasts_df = pd.read_pickle('../pickle_files/english_podcasts_detailed_cleaned.pkl')

In [56]:
podcasts_df['text'] = podcasts_df[['title', 'producer', 'genre', 'description', 'episode_titles', 'episode_descriptions']].apply(lambda x: ' '.join(x), axis=1)
podcasts_df = podcasts_df.drop(columns=['genre', 'description', 'num_episodes', 'rating', 'num_reviews', 'link', 'episode_titles', 'episode_descriptions'])
podcasts_df['ID'] = list(range(podcasts_df.shape[0]))

# Preprocessing Text

In [57]:
# create list of stop words
stop = get_stop_words('en')

# remove non-alphanumeric, non-space
stop = [re.sub(r'([^\s\w]|_)+', '', x) for x in stop]

# add in custom stop words
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

months = ['january', 'february', 'march', 'april', 'may', 'june', 
          'july', 'august', 'september', 'october', 'november', 'december']

other = ['nan', 'podcast', 'podcasts', 'every', 'new', 'weekly', 
         'stories', 'story', 'episode', 'episodes', 'listen', 
         'host', 'hosted', 'join']

[stop.append(str(day)) for day in days]
[stop.append(str(month)) for month in months]
[stop.append(str(x)) for x in other]

def topKFrequent(tokenized_text, k): 
   
    count = Counter(tokenized_text)   
    
    return heapq.nlargest(k, count.keys(), key=count.get)

def remove_stop(text, stop):
    custom_stop = stop
#     top5 = topKFrequent(text, 5)
#     custom_stop = custom_stop + top5
    
    new_text = []
    for word in text:
        if word not in custom_stop:
            new_text.append(word)
    return new_text

# create tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# create stemmer
p_stemmer = PorterStemmer()
l_stemmer = WordNetLemmatizer() 


def stem_list(text, p_stemmer):
    new_list = []
    for word in text:
        new_list.append(p_stemmer.stem(word))
    return new_list

def lem_list(text, l_stemmer):
    new_list = []
    for word in text:
        new_list.append(l_stemmer.lemmatize(word))
    return new_list

def preprocess_text(text):
    # remove mixed alphanumeric
    text = re.sub(r"""(?x) # verbose regex
                            \b    # Start of word
                            (?=   # Look ahead to ensure that this word contains...
                             \w*  # (after any number of alphanumeric characters)
                             \d   # ...at least one digit.
                            )     # End of lookahead
                            \w+   # Match the alphanumeric word
                            \s*   # Match any following whitespace""", 
                             "", text)
    
    # remove urls (will check and remove http and www later)
    text = re.sub(r'\s([\S]*.com[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.org[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.net[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.edu[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.gov[\S]*)\b', '', text)
    
    # remove non-alphanumeric, non-space
    text = re.sub(r'([^\s\w]|_)+', '', text)
    
    # tokenize text
    text = tokenizer.tokenize(text.lower())
    
    # remove stop words
    text = remove_stop(text, stop)
    
    # stem
    text = lem_list(text, l_stemmer)
    
    # remove instances of http or www
    new_text_list = []
    for word in text:
        if re.search(r'http', word):
            continue
        if re.search(r'www', word):
            continue
        new_text_list.append(word)
    
    new_text = ' '.join(new_text_list)
    
    return new_text

In [58]:
podcasts_df['text'] = podcasts_df['text'].map(preprocess_text)
podcasts_df = podcasts_df[podcasts_df.text != '']

In [59]:
podcasts_df.head()

,title,producer,text,ID
0,History Hyenas with Chris Distefano and Yannis...,RiotCast Network,history hyena chris distefano yannis pappa rio...,0
1,Curiosity Daily,Westwood One,curiosity daily westwood one education awardwi...,1
2,Spirits,Multitude,spirit multitude history boozy mythology legen...,2
3,The Soundtrack Show,iHeartRadio,soundtrack show iheartradio tv film soundtrack...,3
4,Writing Excuses,"Brandon Sanderson, Mary Robinette Kowal, Dan W...",writing excuse brandon sanderson mary kowal da...,4


# Podcast Recommendation Models

In [60]:
# helper functions
def get_title_from_index(index):
    return podcasts_df[podcasts_df.ID == index]["title"].values[0]

def get_index_from_title(title):
    return podcasts_df[podcasts_df.title == title]["ID"].values[0]

def get_recommendations(podcast_id, sim_matrix):
    recommendations = list()
    
    podcast_title = get_title_from_index(podcast_id)
    similar_podcasts =  list(enumerate(sim_matrix[podcast_id]))
    sorted_similar_podcast = sorted(similar_podcasts,key=lambda x:x[1],reverse=True)
    
    for i in range(11):
        title = get_title_from_index(sorted_similar_podcast[i][0])
        recommendations.append(title)
    
    return recommendations

test_podcasts = ['The Daily', 'Up First', 'VIEWS with David Dobrik and Jason Nash', 'Impaulsive with Logan Paul',
                 'The Bill Simmons Podcast', 'My Favorite Murder with Karen Kilgariff and Georgia Hardstark',
                 'This American Life', 'Joel Osteen Podcast', 'TED Radio Hour', 'Call Her Daddy', 
                 'Skip and Shannon: Undisputed'
                ]

### Cosine Similarity + CountVectorizer (Bag of Words) Method

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer()
cv_matrix = cv.fit_transform(podcasts_df["text"])
cv_cosine_sim = cosine_similarity(cv_matrix)

In [62]:
for i in test_podcasts:
    print(get_recommendations(get_index_from_title(i), cv_cosine_sim))
    print()

['The Daily', 'Impeachment Inquiry: Updates from The Washington Post', 'Impeachment: A Daily Podcast', 'The Takeaway', 'Article II: Inside Impeachment', "The Daily 202's Big Idea", 'The 11th Hour with Brian Williams', 'Bill O’Reilly’s No Spin News and Analysis', 'The Last Word with Lawrence O’Donnell', 'Up First', 'Impeachment Today']

['Up First', 'Impeachment: A Daily Podcast', 'Impeachment Inquiry: Updates from The Washington Post', 'The Daily', 'Article II: Inside Impeachment', 'The 11th Hour with Brian Williams', 'Bill O’Reilly’s No Spin News and Analysis', "The Daily 202's Big Idea", 'The Takeaway', 'Can He Do That?', 'The Last Word with Lawrence O’Donnell']

['VIEWS with David Dobrik and Jason Nash', 'Instant Message', 'I Am In Eskew', 'Jalen & Jacoby', 'The Axe Files with David Axelrod', 'Getting Things Done', 'The Tower', 'The Permaculture Podcast', 'Making It With Jimmy Diresta, Bob Clagett and David Picciuto', 'Psychology of Eating', 'Blank Check with Griffin & David']

['Im

### Cosine Similarity + TFIDF

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf_matrix = tf.fit_transform(podcasts_df["text"])
tf_cosine_sim = cosine_similarity(tf_matrix)

In [64]:
for i in test_podcasts:
    print(get_recommendations(get_index_from_title(i), tf_cosine_sim))
    print()

['The Daily', 'Impeachment Inquiry: Updates from The Washington Post', "The Daily 202's Big Idea", 'The 11th Hour with Brian Williams', 'Article II: Inside Impeachment', 'Impeachment: A Daily Podcast', 'The Takeaway', 'The Last Word with Lawrence O’Donnell', 'Bill O’Reilly’s No Spin News and Analysis', 'The Situation Room with Wolf Blitzer', 'The Rachel Maddow Show']

['Up First', 'Impeachment Inquiry: Updates from The Washington Post', 'The Daily', 'Article II: Inside Impeachment', 'The 11th Hour with Brian Williams', 'Impeachment: A Daily Podcast', "The Daily 202's Big Idea", 'Can He Do That?', 'The Last Word with Lawrence O’Donnell', 'Bill O’Reilly’s No Spin News and Analysis', 'The Takeaway']

['VIEWS with David Dobrik and Jason Nash', 'Instant Message', 'How Did This Get Made?', 'Light The Fight- Parenting Podcast', 'Flow Sessions with Jason Silva', 'This Week in Startups - Audio', 'The Tower', 'True Cold Case Files with Jason and Daisy', 'Horror Hill: A Horror Anthology and Scary

### Cosine Similarity + Custom Word2Vec Embedding

In [65]:
from gensim.models import Word2Vec

In [66]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [67]:
text_list = list(podcasts_df.text)
tokenized_text = [tokenizer.tokenize(i) for i in text_list]

In [68]:
w2v_model = Word2Vec(tokenized_text)

In [69]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v_model)
mean_embedded = mean_embedding_vectorizer.fit_transform(podcasts_df['text'])
w2v_cosine_sim = cosine_similarity(mean_embedded)

/Users/sidd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


In [70]:
for i in test_podcasts:
    print(get_recommendations(get_index_from_title(i), w2v_cosine_sim))
    print()

['The Daily', 'Impeachment: A Daily Podcast', 'Can He Do That?', 'Up First', 'The Takeaway', 'Global News Podcast', 'Talking Feds', 'Skimm This', "LRC Presents: All the President's Lawyers", 'Bill O’Reilly’s No Spin News and Analysis', 'PBS NewsHour - Full Show']

['Up First', 'The Daily', 'Impeachment: A Daily Podcast', 'Can He Do That?', 'Bill O’Reilly’s No Spin News and Analysis', 'PBS NewsHour - Full Show', 'Global News Podcast', "LRC Presents: All the President's Lawyers", 'PBS NewsHour - Segments', 'Talking Feds', 'Skimm This']

['VIEWS with David Dobrik and Jason Nash', 'The Fighter & The Kid', 'Conan O’Brien Needs A Friend', 'The CONAN Podcast', 'Straight Up with Stassi', "So Bad It's Good with Ryan Bailey", 'Did I Stutter?? with Drew Lynch', 'Ripper Magoos with Bob Menery', 'Amy Schumer Presents: 3 Girls, 1 Keith', 'Adulting', 'You Up with Nikki Glaser']

['Impaulsive with Logan Paul', 'To L And Back: An L Word Podcast', 'Comments by Celebs', 'The Basement Yard', 'Ghost Storie

### Cosine Similarity + GloVe Embedding

In [71]:
from gensim.models import KeyedVectors

In [72]:
glove_model = KeyedVectors.load_word2vec_format("../word2vec/glove.6B.50d.txt.word2vec")

In [73]:
glove_mean_embedding_vectorizer = MeanEmbeddingVectorizer(glove_model)
glove_mean_embedded = glove_mean_embedding_vectorizer.fit_transform(podcasts_df['text'])
glove_cosine_sim = cosine_similarity(glove_mean_embedded)

/Users/sidd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/Users/sidd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/Users/sidd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [74]:
for i in test_podcasts:
    print(get_recommendations(get_index_from_title(i), glove_cosine_sim))
    print()

['The Daily', 'Up First', 'Impeachment: A Daily Podcast', 'The Lawfare Podcast', 'Bag Man', 'Can He Do That?', 'CBS This Morning', 'Impeachment Inquiry: Updates from The Washington Post', 'All In with Chris Hayes', 'The Takeaway', 'The New Yorker: Politics and More']

['Up First', 'The Daily', 'The Lawfare Podcast', 'Impeachment: A Daily Podcast', 'Bag Man', 'What Next | Daily News and Analysis', 'Impeachment Inquiry: Updates from The Washington Post', 'The Situation Room with Wolf Blitzer', 'PBS NewsHour - Segments', 'All In with Chris Hayes', 'The Lead with Jake Tapper']

['VIEWS with David Dobrik and Jason Nash', 'Help! I Suck at Dating with Dean, Vanessa and Jared', 'Amy Schumer Presents: 3 Girls, 1 Keith', 'Dead Pilots Society', "So Bad It's Good with Ryan Bailey", 'The Ben and Ashley I Almost Famous Podcast', 'Out in the Wild', 'Your 2 Dads w/ Sean & Julian', 'Good Morning From Hell', 'The Bobby Bones Show', 'Scrubbing In with Becca Tilley & Tanya Rad']

['Impaulsive with Logan P

### Cosine Similarity + Word2Vec + Smooth Inverse Frequency

In [75]:
from scipy.sparse.linalg import svds
from sklearn.decomposition import TruncatedSVD

In [76]:
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX

def smooth_inverse_frequency(sent, a=0.001, word2vec_model=w2v_model):
    word_counter = {}
    sentences = []
    total_count = 0
    no_of_sentences = 0
    
    for s in sent:
        for w in s:
            if w in word_counter:
                word_counter[w] = word_counter[w] + 1
            else:
                word_counter[w] = 1
        total_count = total_count + len(s)
        no_of_sentences = no_of_sentences + 1
    
    sents_emd = []
    for s in sent:
        sent_emd = []
        for word in s:
            if word in word2vec_model:
                emd = (a/(a + (word_counter[word]/total_count)))*word2vec_model[word]
                sent_emd.append(emd)
        sum_ = np.array(sent_emd).sum(axis=0)
        sentence_emd = sum_/float(no_of_sentences)
        sents_emd.append(sentence_emd)
        
    new_sents_emb = remove_first_principal_component(np.array(sents_emd))
    return new_sents_emb

In [77]:
sif_text_emd = smooth_inverse_frequency(text_list)
sif_cosine_sim = cosine_similarity(sif_text_emd)

/Users/sidd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/Users/sidd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [78]:
for i in test_podcasts:
    print(get_recommendations(get_index_from_title(i), sif_cosine_sim))
    print()

['The Daily', 'Impeachment Inquiry: Updates from The Washington Post', 'Criminal', 'Queens Podcast', 'Omnarchy', 'La Corneta', 'EEs Talk Tech - An Electrical Engineering Podcast', 'The Quilting Company Podcast', 'Constitutional', 'The Drum Show', 'Quantum Physics I']

['Up First', 'The Paul Tripp Podcast', 'Best of Both Worlds Podcast', 'The General Hospital Podcast', "Heaven's Gate", 'Wild Thing', 'Post Reports', 'Stay Tuned with Preet', 'Article II: Inside Impeachment', "Quentin Tarantino's Feature Presentation", 'The Chris Hogan Show']

['VIEWS with David Dobrik and Jason Nash', 'The Bunker', 'Beach Too Sandy, Water Too Wet', 'Solid Joys Daily Devotional', 'Bald Move TV', "WHOA That's Good Podcast", 'MarriageToday Audio Podcast', 'Survivor: Island of the Idols - Recaps from Rob has a Podcast | RHAP', 'FieldCraft Survival', 'KEXP Presents Music That Matters', 'Instant Message']

['Impaulsive with Logan Paul', 'The Tom Ferry Podcast Experience', 'Purchasing Truth', 'Yogaland Podcast',

### WMD (Package Not Working)

In [ ]:
from pyemd import emd
from gensim.similarities import WmdSimilarity
glove_model.init_sims(replace=True)

In [ ]:
D = np.zeros((len(text_list), len(text_list)))
D.shape

In [ ]:
for i in range(len(text_list)):
    for j in range(len(text_list)):
        if i == j:
            continue  # self-distance is 0.0
        if i > j:
            D[i, j] = D[j, i]  # re-use earlier calc
        D[i, j] = glove_model.wmdistance(text_list[i], text_list[j])

In [ ]:
glove_cosine_sim.shape

In [ ]:
text_list[0]

In [ ]:
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_sif_benchmark(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=0.001):
    total_freq = sum(freqs.values())

    embeddings = []

    # SIF requires us to first collect all sentence embeddings and then perform
    # common component analysis.
    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
        weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

        embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)

        embeddings.append(embedding1)
        embeddings.append(embedding2)

    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
            for idx in range(int(len(embeddings) / 2))]

    return sims